In [1]:
from pymongo import MongoClient
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

In [2]:
import re
comp= {"name":{"$exists":True}}
cur = db.companies.find(comp)
comp2 = list(cur)

proyection ={"offices":1, "name":1,"category_code":1,"number_of_employees":1,"founded_year":1,"total_money_raised":1,}

comp_result= list(db.companies.find(comp,proyection))



In [3]:
import pandas as pd

df = pd.DataFrame(list(comp_result))
df

,_id,name,category_code,number_of_employees,founded_year,total_money_raised,offices
0,52cdef7c4bab8bd675297d8b,AdventNet,enterprise,600.0,1996.0,$0,"[{'description': 'Headquarters', 'address1': '..."
1,52cdef7c4bab8bd675297d8c,Zoho,software,1600.0,2005.0,$0,"[{'description': 'Headquarters', 'address1': '..."
2,52cdef7c4bab8bd675297d8f,Omnidrive,network_hosting,NaN,2005.0,$800k,"[{'description': '', 'address1': 'Suite 200', ..."
3,52cdef7c4bab8bd675297d8a,Wetpaint,web,47.0,2005.0,$39.8M,"[{'description': '', 'address1': '710 - 2nd Av..."
4,52cdef7c4bab8bd675297d90,Postini,web,NaN,1999.0,$0,"[{'description': None, 'address1': '959 Skyway..."
...,...,...,...,...,...,...,...
18796,52cdef7f4bab8bd67529c6f8,goBookmaker,web,NaN,NaN,$0,[]
18797,52cdef7f4bab8bd67529c6f9,EnteGreat Solutions,software,NaN,NaN,$0,"[{'description': '', 'address1': '', 'address2..."
18798,52cdef7f4bab8bd67529c6fa,EnteGreat,enterprise,NaN,2000.0,$0,"[{'description': '', 'address1': '1900 Interna..."
18799,52cdef7f4bab8bd67529c6f7,AfterLogic,software,NaN,2002.0,$0,"[{'description': 'Livingston', 'address1': 'P...."


In [4]:
offices = df.explode("offices").apply(lambda e: e.offices,axis=1,result_type="expand")
offices

,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
2,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
3,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
3,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
...,...,...,...,...,...,...,...,...,...
18796,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18797,,,,,Birmingham,AL,USA,33.518885,-86.816068
18798,,1900 International Park Drive,Suite 200,35243-4204,Birmingham,AL,USA,33.407786,-86.761302
18799,Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,40.793024,-74.323554


In [5]:
clean_offices = pd.concat([df[["name","_id","total_money_raised","number_of_employees","category_code","founded_year"]], offices], axis=1)
clean_offices

,name,_id,total_money_raised,number_of_employees,category_code,founded_year,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,AdventNet,52cdef7c4bab8bd675297d8b,$0,600.0,enterprise,1996.0,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,Zoho,52cdef7c4bab8bd675297d8c,$0,1600.0,software,2005.0,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
2,Omnidrive,52cdef7c4bab8bd675297d8f,$800k,NaN,network_hosting,2005.0,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
3,Wetpaint,52cdef7c4bab8bd675297d8a,$39.8M,47.0,web,2005.0,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
3,Wetpaint,52cdef7c4bab8bd675297d8a,$39.8M,47.0,web,2005.0,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18796,goBookmaker,52cdef7f4bab8bd67529c6f8,$0,NaN,web,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18797,EnteGreat Solutions,52cdef7f4bab8bd67529c6f9,$0,NaN,software,NaN,,,,,Birmingham,AL,USA,33.518885,-86.816068
18798,EnteGreat,52cdef7f4bab8bd67529c6fa,$0,NaN,enterprise,2000.0,,1900 International Park Drive,Suite 200,35243-4204,Birmingham,AL,USA,33.407786,-86.761302
18799,AfterLogic,52cdef7f4bab8bd67529c6f7,$0,NaN,software,2002.0,Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,40.793024,-74.323554


In [6]:
(clean_offices).head(1000)

,name,_id,total_money_raised,number_of_employees,category_code,founded_year,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
0,AdventNet,52cdef7c4bab8bd675297d8b,$0,600.0,enterprise,1996.0,Headquarters,4900 Hopyard Rd.,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
1,Zoho,52cdef7c4bab8bd675297d8c,$0,1600.0,software,2005.0,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
2,Omnidrive,52cdef7c4bab8bd675297d8f,$800k,NaN,network_hosting,2005.0,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
3,Wetpaint,52cdef7c4bab8bd675297d8a,$39.8M,47.0,web,2005.0,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
3,Wetpaint,52cdef7c4bab8bd675297d8a,$39.8M,47.0,web,2005.0,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,Vitrue,52cdef7c4bab8bd675298095,$33M,NaN,enterprise,2006.0,Headquarters,101 Marietta Street,Suite 1700,30303,Atlanta,GA,USA,33.756806,-84.392198
779,Vitrue,52cdef7c4bab8bd675298095,$33M,NaN,enterprise,2006.0,NYC Office,205 Lexington Ave.,Suite 700,10016,New York,NY,USA,40.718888,-74.001169
779,Vitrue,52cdef7c4bab8bd675298095,$33M,NaN,enterprise,2006.0,San Francisco,650 5th St.,Suite 303,94107,San Francisco,CA,USA,NaN,NaN
779,Vitrue,52cdef7c4bab8bd675298095,$33M,NaN,enterprise,2006.0,Chicago,30 W Hubbard St.,Suite 400,60654,Chicago,IL,USA,NaN,NaN


In [7]:
clean_offices["category_code"].value_counts()

web                 4054
software            3392
mobile              1302
games_video         1200
advertising         1183
other               1155
enterprise           946
consulting           807
ecommerce            777
network_hosting      699
public_relations     619
search               496
hardware             461
biotech              402
cleantech            359
semiconductor        192
security             190
analytics             94
finance               66
news                  64
social                63
legal                 51
music                 50
messaging             47
education             43
photo_video           31
travel                29
health                28
medical               27
manufacturing         22
transportation        16
hospitality           14
sports                14
automotive            11
fashion               10
real_estate           10
design                 6
nanotech               5
nonprofit              4
local                  1


In [8]:
categories = ["web","software","mobile","games_video","advertising","ecommerce","network_hosting","search","music"]

In [9]:
usa2 = clean_offices[(clean_offices["category_code"].isin(categories))]

In [10]:
usa2

,name,_id,total_money_raised,number_of_employees,category_code,founded_year,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
1,Zoho,52cdef7c4bab8bd675297d8c,$0,1600.0,software,2005.0,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
2,Omnidrive,52cdef7c4bab8bd675297d8f,$800k,NaN,network_hosting,2005.0,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
3,Wetpaint,52cdef7c4bab8bd675297d8a,$39.8M,47.0,web,2005.0,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
3,Wetpaint,52cdef7c4bab8bd675297d8a,$39.8M,47.0,web,2005.0,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
4,Postini,52cdef7c4bab8bd675297d90,$0,NaN,web,1999.0,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18795,Adhunk,52cdef7f4bab8bd67529c6f6,$0,3.0,advertising,2009.0,Indian Office,Near Tube Well(south),Patel Nagar,334003,Bikaner,None,IND,NaN,NaN
18796,goBookmaker,52cdef7f4bab8bd67529c6f8,$0,NaN,web,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18797,EnteGreat Solutions,52cdef7f4bab8bd67529c6f9,$0,NaN,software,NaN,,,,,Birmingham,AL,USA,33.518885,-86.816068
18799,AfterLogic,52cdef7f4bab8bd67529c6f7,$0,NaN,software,2002.0,Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,40.793024,-74.323554


In [11]:
usa2.dtypes

name                    object
_id                     object
total_money_raised      object
number_of_employees    float64
category_code           object
founded_year           float64
description             object
address1                object
address2                object
zip_code                object
city                    object
state_code              object
country_code            object
latitude               float64
longitude              float64
dtype: object

In [12]:
usa2["total_money_raised"] = usa2["total_money_raised"].replace({'\$':''}, regex = True , )

<ipython-input-12-b7febea41167>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa2["total_money_raised"] = usa2["total_money_raised"].replace({'\$':''}, regex = True , )


In [13]:
usa2

,name,_id,total_money_raised,number_of_employees,category_code,founded_year,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude
1,Zoho,52cdef7c4bab8bd675297d8c,0,1600.0,software,2005.0,Headquarters,4900 Hopyard Rd,Suite 310,94588,Pleasanton,CA,USA,37.692934,-121.904945
2,Omnidrive,52cdef7c4bab8bd675297d8f,800k,NaN,network_hosting,2005.0,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN
3,Wetpaint,52cdef7c4bab8bd675297d8a,39.8M,47.0,web,2005.0,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253
3,Wetpaint,52cdef7c4bab8bd675297d8a,39.8M,47.0,web,2005.0,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431
4,Postini,52cdef7c4bab8bd675297d90,0,NaN,web,1999.0,None,"959 Skyway Road, Suite 200",None,94070,San Carlos,CA,USA,37.506885,-122.247573
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18795,Adhunk,52cdef7f4bab8bd67529c6f6,0,3.0,advertising,2009.0,Indian Office,Near Tube Well(south),Patel Nagar,334003,Bikaner,None,IND,NaN,NaN
18796,goBookmaker,52cdef7f4bab8bd67529c6f8,0,NaN,web,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18797,EnteGreat Solutions,52cdef7f4bab8bd67529c6f9,0,NaN,software,NaN,,,,,Birmingham,AL,USA,33.518885,-86.816068
18799,AfterLogic,52cdef7f4bab8bd67529c6f7,0,NaN,software,2002.0,Livingston,P.O. Box 2096,,07039,Livingston,NJ,USA,40.793024,-74.323554


In [14]:
usa2["total_money_raised"] = usa2["total_money_raised"].replace({'\€':''}, regex = True )

<ipython-input-14-e742c9421109>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa2["total_money_raised"] = usa2["total_money_raised"].replace({'\€':''}, regex = True )


In [15]:
usa2["total_money_raised"] = usa2["total_money_raised"].replace({'\£':''}, regex = True )

<ipython-input-15-7cff9354b1b9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa2["total_money_raised"] = usa2["total_money_raised"].replace({'\£':''}, regex = True )


In [16]:
usa2['LastDigit'] = usa2['total_money_raised'].str.strip().str[-1]


<ipython-input-16-1daa351a2e5e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa2['LastDigit'] = usa2['total_money_raised'].str.strip().str[-1]


In [17]:
usa2["LastDigit"].value_counts()

0    9205
M    3285
k     655
B       8
Name: LastDigit, dtype: int64

In [18]:
cat= ['k','M','B']
usa3= usa2[(usa2["LastDigit"].isin(cat))]

In [19]:
usa3["LastDigit"].value_counts()

M    3285
k     655
B       8
Name: LastDigit, dtype: int64

In [20]:
usa3

,name,_id,total_money_raised,number_of_employees,category_code,founded_year,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,LastDigit
2,Omnidrive,52cdef7c4bab8bd675297d8f,800k,NaN,network_hosting,2005.0,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN,k
3,Wetpaint,52cdef7c4bab8bd675297d8a,39.8M,47.0,web,2005.0,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253,M
3,Wetpaint,52cdef7c4bab8bd675297d8a,39.8M,47.0,web,2005.0,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431,M
6,Geni,52cdef7c4bab8bd675297d91,16.5M,18.0,web,2006.0,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064,M
9,StumbleUpon,52cdef7c4bab8bd675297d95,18.5M,NaN,web,2002.0,,,,,San Francisco,CA,USA,37.775196,-122.419204,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18733,Tiny Speck,52cdef7f4bab8bd67529c6b6,17.2M,45.0,games_video,2009.0,,163 Hastings St W,,BC V6B 1H5,Vancouver,None,CAN,49.282455,-123.109217,M
18768,iGuiders,52cdef7f4bab8bd67529c6d9,1.17M,5.0,web,2008.0,,23811 Chagrin,Boulevard Suite 244,44122,Beachwood,OH,USA,NaN,NaN,M
18779,Getyoo,52cdef7f4bab8bd67529c6e4,1.13M,10.0,mobile,2009.0,HQ,"Rue des Tanneurs, 60a",,1000,Brussels,None,BEL,50.839970,4.346472,M
18782,OfficialVirtualDJ,52cdef7f4bab8bd67529c6e6,25M,102.0,music,2009.0,Created in London,11 Eggleston view,,DL3 9SH,London,DC,GBR,NaN,NaN,M


In [21]:
usa3['first_digit'] = usa3['total_money_raised'].str.strip().str[0]



<ipython-input-21-20332acad449>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa3['first_digit'] = usa3['total_money_raised'].str.strip().str[0]


In [22]:
usa3

,name,_id,total_money_raised,number_of_employees,category_code,founded_year,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,LastDigit,first_digit
2,Omnidrive,52cdef7c4bab8bd675297d8f,800k,NaN,network_hosting,2005.0,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN,k,8
3,Wetpaint,52cdef7c4bab8bd675297d8a,39.8M,47.0,web,2005.0,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253,M,3
3,Wetpaint,52cdef7c4bab8bd675297d8a,39.8M,47.0,web,2005.0,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431,M,3
6,Geni,52cdef7c4bab8bd675297d91,16.5M,18.0,web,2006.0,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064,M,1
9,StumbleUpon,52cdef7c4bab8bd675297d95,18.5M,NaN,web,2002.0,,,,,San Francisco,CA,USA,37.775196,-122.419204,M,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18733,Tiny Speck,52cdef7f4bab8bd67529c6b6,17.2M,45.0,games_video,2009.0,,163 Hastings St W,,BC V6B 1H5,Vancouver,None,CAN,49.282455,-123.109217,M,1
18768,iGuiders,52cdef7f4bab8bd67529c6d9,1.17M,5.0,web,2008.0,,23811 Chagrin,Boulevard Suite 244,44122,Beachwood,OH,USA,NaN,NaN,M,1
18779,Getyoo,52cdef7f4bab8bd67529c6e4,1.13M,10.0,mobile,2009.0,HQ,"Rue des Tanneurs, 60a",,1000,Brussels,None,BEL,50.839970,4.346472,M,1
18782,OfficialVirtualDJ,52cdef7f4bab8bd67529c6e6,25M,102.0,music,2009.0,Created in London,11 Eggleston view,,DL3 9SH,London,DC,GBR,NaN,NaN,M,2


In [23]:
exclude =["1","2","3","4","5","6","7","8","9"]
usa3= usa3[(usa3["first_digit"].isin(exclude))]

In [24]:
usa3

,name,_id,total_money_raised,number_of_employees,category_code,founded_year,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,LastDigit,first_digit
2,Omnidrive,52cdef7c4bab8bd675297d8f,800k,NaN,network_hosting,2005.0,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN,k,8
3,Wetpaint,52cdef7c4bab8bd675297d8a,39.8M,47.0,web,2005.0,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253,M,3
3,Wetpaint,52cdef7c4bab8bd675297d8a,39.8M,47.0,web,2005.0,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431,M,3
6,Geni,52cdef7c4bab8bd675297d91,16.5M,18.0,web,2006.0,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064,M,1
9,StumbleUpon,52cdef7c4bab8bd675297d95,18.5M,NaN,web,2002.0,,,,,San Francisco,CA,USA,37.775196,-122.419204,M,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18733,Tiny Speck,52cdef7f4bab8bd67529c6b6,17.2M,45.0,games_video,2009.0,,163 Hastings St W,,BC V6B 1H5,Vancouver,None,CAN,49.282455,-123.109217,M,1
18768,iGuiders,52cdef7f4bab8bd67529c6d9,1.17M,5.0,web,2008.0,,23811 Chagrin,Boulevard Suite 244,44122,Beachwood,OH,USA,NaN,NaN,M,1
18779,Getyoo,52cdef7f4bab8bd67529c6e4,1.13M,10.0,mobile,2009.0,HQ,"Rue des Tanneurs, 60a",,1000,Brussels,None,BEL,50.839970,4.346472,M,1
18782,OfficialVirtualDJ,52cdef7f4bab8bd67529c6e6,25M,102.0,music,2009.0,Created in London,11 Eggleston view,,DL3 9SH,London,DC,GBR,NaN,NaN,M,2


In [25]:
multipliers = {'k':1000, 'M':1000000, 'B':1000000000}

In [26]:
count=0
for i in usa3.total_money_raised:
    if i[-1].isdigit(): # check if no suffix
        print(i)
        usa3["total_money_raised"].iloc[count] = int(string)
        count+=1
    else:
        mult = multipliers[i[-1]] # look up suffix to get multiplier
        usa3["total_money_raised"].iloc[count] = int(float(i[:-1]) * mult)
        count+=1
    

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
<ipython-input-26-e3baf631bf5d>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  usa3["total_money_raised"].iloc[count] = int(float(i[:-1]) * mult)


In [27]:
usa3.total_money_raised.iloc[10]

13200000

In [ ]:
#def string_to_int(string):
#    if string[-1].isdigit(): # check if no suffix
#        return int(string)
#    mult = multipliers[string[-1]] # look up suffix to get multiplier
#    return int(float(string[:-1]) * mult)# convert number to float, multiply by multiplier, then make int


#print(list(map(string_to_int, usa3.total_money_raised)))

In [28]:
usa3

,name,_id,total_money_raised,number_of_employees,category_code,founded_year,description,address1,address2,zip_code,city,state_code,country_code,latitude,longitude,LastDigit,first_digit
2,Omnidrive,52cdef7c4bab8bd675297d8f,800000,NaN,network_hosting,2005.0,,Suite 200,654 High Street,94301,Palo Alto,CA,ISR,NaN,NaN,k,8
3,Wetpaint,52cdef7c4bab8bd675297d8a,39800000,47.0,web,2005.0,,710 - 2nd Avenue,Suite 1100,98104,Seattle,WA,USA,47.603122,-122.333253,M,3
3,Wetpaint,52cdef7c4bab8bd675297d8a,39800000,47.0,web,2005.0,,270 Lafayette Street,Suite 505,10012,New York,NY,USA,40.723731,-73.996431,M,3
6,Geni,52cdef7c4bab8bd675297d91,16500000,18.0,web,2006.0,Headquarters,9229 W. Sunset Blvd.,,90069,West Hollywood,CA,USA,34.090368,-118.393064,M,1
9,StumbleUpon,52cdef7c4bab8bd675297d95,18500000,NaN,web,2002.0,,,,,San Francisco,CA,USA,37.775196,-122.419204,M,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18733,Tiny Speck,52cdef7f4bab8bd67529c6b6,17200000,45.0,games_video,2009.0,,163 Hastings St W,,BC V6B 1H5,Vancouver,None,CAN,49.282455,-123.109217,M,1
18768,iGuiders,52cdef7f4bab8bd67529c6d9,1170000,5.0,web,2008.0,,23811 Chagrin,Boulevard Suite 244,44122,Beachwood,OH,USA,NaN,NaN,M,1
18779,Getyoo,52cdef7f4bab8bd67529c6e4,1130000,10.0,mobile,2009.0,HQ,"Rue des Tanneurs, 60a",,1000,Brussels,None,BEL,50.839970,4.346472,M,1
18782,OfficialVirtualDJ,52cdef7f4bab8bd67529c6e6,25000000,102.0,music,2009.0,Created in London,11 Eggleston view,,DL3 9SH,London,DC,GBR,NaN,NaN,M,2
